# Create JSONL for vertex AI autoML

In [12]:
!pip install google-cloud-storage


You should consider upgrading via the '/Users/louis.sanna/.pyenv/versions/3.8.16/bin/python3.8 -m pip install --upgrade pip' command.


In [13]:

# Replace with your own bucket name and prefix
bucket_name = "fish-dataset"
prefix = "first-selection"
csv_file_path = "../data/raw/fish-species.csv"

import csv
import json
from google.cloud import storage

def read_csv(file_path):
    id_to_name = {}
    with open(file_path, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            id_to_name[row['ID']] = row['nom_commun']
    return id_to_name

def list_images(bucket_name, prefix):
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blobs = list(bucket.list_blobs(prefix=prefix))

    image_paths = [blob.public_url for blob in blobs if blob.name.lower().endswith(('.jpg', '.jpeg', '.png'))]
    return image_paths

def get_labels_from_paths(image_paths, id_to_name):
    labels = [id_to_name[path.split('/')[-2]] for path in image_paths]
    return labels

def create_jsonl(image_paths, labels):
    with open('../data/processed/data.jsonl', 'w') as jsonl_file:
        for path, label in zip(image_paths, labels):
            data = {
                "imageGcsUri": path.replace("https://storage.googleapis.com/", "gs://"),
                "classificationAnnotation": {"displayName": label}
            }
            jsonl_file.write(json.dumps(data) + '\n')

id_to_name = read_csv(csv_file_path)
image_paths = list_images(bucket_name, prefix)
labels = get_labels_from_paths(image_paths, id_to_name)

create_jsonl(image_paths, labels)


/Users/louis.sanna/.pyenv/versions/3.8.16/lib/python3.8/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/louis.sanna/.pyenv/versions/3.8.16/lib/python3.8/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
